In [1]:
import requests
import pymongo

In [2]:
from dotenv import load_dotenv
load_dotenv() 
import os

FINNHUB_KEY = os.environ['FINNHUB_KEY']
FINNHUB_URI = os.environ['FINNHUB_URI']

print(FINNHUB_URI)

https://finnhub.io/api/v1/


In [3]:
#save all book links to db
import pymongo
# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Access a database
db = client["stock_forum"]

# Access a collection
tbl_stock = db["tbl_stock"]

In [4]:
def custom_query(get_url):
    print(get_url)
    try:
        r = requests.get(get_url)
        return r.json()
    except Exception as e:
       print(e)
       return r

In [5]:
#1. Get all stock symbols
stock_list_url = FINNHUB_URI + 'stock/symbol?exchange=US&token=' + FINNHUB_KEY

stock_list = custom_query(stock_list_url)
print(stock_list[0])

https://finnhub.io/api/v1/stock/symbol?exchange=US&token=cu5b37pr01qqj8u65ufgcu5b37pr01qqj8u65ug0
{'currency': 'USD', 'description': 'TERRACE ENERGY CORP', 'displaySymbol': 'TCRRF', 'figi': 'BBG000WCQW17', 'isin': None, 'mic': 'OOTC', 'shareClassFIGI': 'BBG001STBC13', 'symbol': 'TCRRF', 'symbol2': '', 'type': 'Common Stock'}


In [8]:
db_stocks = tbl_stock.find({})
print(db_stocks[0])

{'_id': ObjectId('678ab5f520c6690d5c97bd91'), 'currency': 'USD', 'description': 'SORTIS HOLDINGS INC', 'mic': 'OOTC', 'symbol': '111', 'type': 'Common Stock', 'about': 'long_text 111', 'wachers': 34567}


In [ ]:
#2. save basic info to db -> 2 mins
for stock in stock_list:
    if stock['mic'].find('OTC') >= 0:    #this app is used for OTC market
        #upsert to db
        db_stock_detail = tbl_stock.find_one({'symbol': stock['symbol']})
        if db_stock_detail is None:
            #insert
            tbl_stock.insert_one({
                'symbol': stock['symbol'],
                "currency" : stock['currency'],
                "description" : stock['description'],
                "mic" : stock['mic'],
                "type" : stock['type']
            })
        else:
            #update
            tbl_stock.update_one({'symbol': stock['symbol']}, {'$set':{
                "currency" : stock['currency'],
                "description" : stock['description'],
                "mic" : stock['mic'],
                "type" : stock['type']
            }})